In [1]:
using TaylorSeries, TaylorIntegration
using Plots, LaTeXStrings
# plotly()
pyplot(grid = :false, legend = :false, size=(600,600), color=:black, markersize = 0.3 )
const Z = 2.0
const order = 20
const t0 = 0.0
const t_max = 0.5 
const abs_tol = 1.0E-20
const steps = 500_000

500000

In [2]:
function undrivenHelium(τ, q)
    tT, Q₁, Q₂, P₁, P₂ = q
    
    t1 = Q₁^2
    t2 = Q₂^2
    t = t1 * t2
    R12 = t1 - t2
    q₁ = 0.25*t2*P₁
    p₁ = 2Q₁*(-0.125P₂^2 + Z  - t2 - t2/abs(R12) + t*R12/abs(R12)^3)
    q₂ = 0.25*t1*P₂
    p₂ = 2Q₂*(-0.125P₁^2 + Z  - t1 - t1/abs(R12) - t*R12/abs(R12)^3)

    return [t, q₁, q₂, p₁, p₂] 

end
undrivenHelium(v) = undrivenHelium(zero(eltype(v)), v)


var1 = set_variables("t q1 q2 p1 p2", order = 1)


function condini(x10::Float64, px10::Float64)
    @assert x10 != 0
    Q1 = sqrt(x10)
    Q2 = 0.0
    P1 = px10*sqrt(x10)    
    P2 = sqrt(8Z)    
    return [0, Q1, Q2, P1, P2]
end

function regHam(t, Q₁, Q₂, P₁, P₂)
    #Cantidades auxiliares
    P₁² = P₁^2
    P₂² = P₂^2
    Q₁² = Q₁^2
    Q₂² = Q₂^2
    nf = abs(Q₁² - Q₂²)
    
    H = 0.125*(P₁²*Q₂² + P₂²*Q₁²) - Z*(Q₁² + Q₂²) + Q₁²*Q₂²*(1.0 + 1.0/nf)
    return H
end


regHam(v) = regHam(v...)

function errHam(N::Int)
    srand(487293456)
    J = [0 0 0 0 0; 0 0 0 1 0; 0 0 0 0 1; 0 -1 0 0 0; 0 0 -1 0 0 ]
    
    dnorm = zeros(N)
    als = typeof(zeros(5))[]
    for j in 1:N
        al = 10*rand(5)
        #al = [BigFloat(x) for x in al1]
        alt = al + var1
        ene = J*∇(regHam(alt))
        ene1 = [ene[k].coeffs[1].coeffs[1] for k in 1:5]
        meq = undrivenHelium(al)
        push!(als, al)
        meq[1] = 0
        dnorm[j] = norm(meq - ene1) 
    end
    return dnorm, als 
end

"""
    stabilitymatrix!{T<:Number, S<:Number}(eqsdiff, t0::T, x::Array{T,1},
        jjac::Array{T,2})
    stabilitymatrix!{T<:Number, S<:Number}(eqsdiff, t0::T, x::Array{Taylor1{T},1},
        jjac::Array{Taylor1{T},2})
Updates the matrix `jjac` (linearized equations of motion)
computed from the equations of motion (`eqsdiff`), at time `t0`
at `x0`.
"""
function stabilitymatrix!{T<:Number}(eqsdiff, t0::T, x::Array{T,1},
        jjac::Array{T,2})
    δx = Array{TaylorN{T}}( length(x) )
    @inbounds for ind in eachindex(x)
        δx[ind] = x[ind] + TaylorN(T,ind,order=1)
    end
    jjac[:] = jacobian( eqsdiff(t0, δx), x )
    nothing
end

stabilitymatrix!

In [3]:
function q2sign(xold, x0, tol)
    if abs(xold[3]) < tol
        sq2o = 0
    else
        sq2o = sign(xold[3])
    end
    
    if abs(x0[3]) < tol
        sq2 = 0
    else
        sq2 = sign(x0[3])
    end
    if abs(xold[5]) < tol
        sp2o = 0
    else
        sp2o = sign(xold[5])
    end
    
    if abs(x0[5]) < tol
        sp2 = 0
    else
        sp2 = sign(x0[5])
    end
    return sq2o, sq2, sp2o, sp2
end
    

q2sign (generic function with 1 method)

In [4]:
###Implementamos nuevo código 
function taylorstep1!{T<:Number}(f, t0::T, t1::T, x0::Array{T,1},
        order::Int, abstol::T)
    @assert t1 > t0
    # Initialize the vector of Taylor1 expansions
    xT = Array{Taylor1{TaylorN{T}}}(length(x0))
    for i in eachindex(x0)
        @inbounds xT[i] = Taylor1( x0[i], order )
    end
    # Compute the Taylor coefficients
    jetcoeffs!(f, t0, xT)
    # Compute the step-size of the integration using `abstol`
    δt = TaylorIntegration.stepsize(xT, abstol)
    if δt ≥ t1-t0
        δt = t1-t0
    end
    evaluate!(xT, δt, x0)
    return δt, xT
end


function mytaylorinteg{T<:Number}(f, q0::Array{TaylorN{T},1}, t0::T, tmax::T, order::Int,
    abstol::T; tsteps::Int = 1000, tol::Float64=1e-20, maxsteps::Int=5000)

    # Allocation
    tv = Array{T}(maxsteps)
    dof = length(q0)
    xv = Array{T}(dof, maxsteps)
    jjac0 = eye(dof)
    jjac0[1] = zero(T)
    arrjac = Array{Array{T,2}}(maxsteps)
    sm0 = eye(dof)
    sm0[1] = zero(T)
    stabilitymatrix!(undrivenHelium, t0, q0, sm0)
    # Initial conditions
    @inbounds tv[1] = t0
    @inbounds xv[:,1] = q0[:]
    arrjac[1] = jjac0
    x0 = copy(q0)
    sum1 = q0[3]/maxsteps

    # Integration
    nsteps = 1
    δtn = Inf
    while t0 < tmax
        xold = copy(x0)
        δt, sT = taylorstep1!(f, t0, tmax, x0, order, abstol)
        sq2o, sq2, sp2o, sp2 = q2sign(xold, x0, tol)
        steps1 = 0
        x00 = copy(x0)
        bool1 = false
        
        while sq2o*sq2 == -1 || (sp2o*sp2 == -1 && abs(xold[3]) < tol)
            bool1 = true
            q2T = sT[3]
            dq2T = derivative(q2T)
            δtn = copy(δt)
            for nc in 1:20
                δtn = δtn - evaluate(q2T, δtn)/evaluate(dq2T, δtn)
            end
            
            evaluate!(sT, δtn, x0)  
            sq2o, sq2, sp2o, sp2 = q2sign(xold, x0, tol)
            steps1 += 1
            if steps1  ≥ tsteps
                break
            end
        end
                
        if nsteps ≥ maxsteps 
            warn("""
            Maximum number of integration steps reached; exiting.
            """)
            break
        end
        
                
        nsteps += 1
        
        if bool1 == true
            @inbounds tv[nsteps] = t0 + δtn
            @inbounds xv[:,nsteps] = x0[:]
            stabilitymatrix!(undrivenHelium, t0 + δtn, x0, sm0)
            jjac0 = sm0 * arrjac[nsteps - 1]
            arrjac[nsteps] = jjac0
            x0 = x00
            nsteps += 1
            stabilitymatrix!(undrivenHelium, t0 + δt, x0, sm0)
            jjac0 = sm0 * arrjac[nsteps - 2]
            arrjac[nsteps] = jjac0
        else
            stabilitymatrix!(undrivenHelium, t0 + δt, x0, sm0)
            jjac0 = sm0 * arrjac[nsteps - 1]
            arrjac[nsteps] = jjac0            
        end
        t0 += δt
        @inbounds tv[nsteps] = t0
        @inbounds xv[:,nsteps] = x0[:]
        sum1 += x0[3]/maxsteps
            
    end

    return view(tv,1:nsteps), view(xv,:,1:nsteps)', arrjac[1:nsteps]
end

mytaylorinteg (generic function with 1 method)

In [28]:
c1 = condini(2.0, 0.0) + var1

5-element Array{TaylorSeries.TaylorN{Float64},1}:
                        1.0 t + 𝒪(‖x‖²)
  1.4142135623730951 + 1.0 q1 + 𝒪(‖x‖²)
                       1.0 q2 + 𝒪(‖x‖²)
                       1.0 p1 + 𝒪(‖x‖²)
                 4.0 + 1.0 p2 + 𝒪(‖x‖²)

In [6]:
@time t1, x1 = taylorinteg(undrivenHelium, c1, t0, 2.0, order, abs_tol, maxsteps=100_000);

  0.480590 seconds (558.19 k allocations: 37.248 MB, 3.21% gc time)


In [7]:
@time t2, x2, m1 = mytaylorinteg(undrivenHelium, c1, t0, 2.0, order, abs_tol, maxsteps=100)

  1.293237 seconds (6.68 M allocations: 334.574 MB, 5.96% gc time)


([0.0,0.0688297,0.132565,0.190324,0.243784,0.297682,0.340833,0.379243,0.414926,0.45021  …  1.77388,1.80112,1.82789,1.85322,1.88136,1.90655,1.93201,1.95891,1.98522,2.0],
[0.0 1.41421 … 0.0 4.0; 0.000868728 1.41421 … -0.000607237 3.99044; … ; 3.71783 2.12447 … 3.76379 1.15745; 3.813 2.14422 … 3.80081 1.38402],

Array{Float64,2}[
[0.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0],

[0.0 0.053514 … 0.0 0.0; 0.0 0.0 … 0.00473001 0.0; … ; 0.0 -0.0192936 … 0.0 -2.82167; 0.0 -0.770528 … 4.17628e-5 0.0],

[0.0 2.98349 … 0.000934817 0.528675; 0.0 -0.00189827 … 0.0 -0.0495674; … ; 0.0 -1.89572 … -0.000474582 -0.718598; 0.0 -6.93677 … -0.00679796 -1.23076],

[0.0 -0.583657 … 0.0200887 -0.0200581; 0.0 -0.0669536 … -4.63567e-5 -0.0257028; … ; 0.0 19.9893 … -0.00701735 3.42185; 0.0 1.1688 … -0.0402998 0.0949883],

[0.0 -6.74583 … -0.00658821 -1.4694; 0.0 1.1818 … -0.000389637 0.20317; … ; 0.0 4.7063 … 0.11789 1.45645; 0.0 13.8437 … 0.0133574 3.02077],

[0.0 2.08569 … -0.

In [9]:
m1[64]

5×5 Array{Float64,2}:
 0.0  -1.58666e36  -3.59576e35   2.37833e33  -3.09128e35
 0.0  -4.02395e35  -2.90454e34  -2.22947e31  -8.1733e34 
 0.0   1.7462e35   -3.02455e35   3.18053e33   5.23722e34
 0.0   7.09354e35   4.82067e35  -4.29705e33   1.20964e35
 0.0  -3.39184e36  -4.55747e35   1.93483e33  -6.77622e35

In [8]:
for j in 1:64
    a = (m1[j][2:end,2:end])
    println(a)
end

[1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]
[0.0 -4.17628e-5 0.00473001 0.0; 2.82167 0.0 0.0 0.5; -0.0192936 -0.770528 0.0 -2.82167; -0.770528 -2.11625 4.17628e-5 0.0]
[-0.00189827 -0.0134601 0.0 -0.0495674; -0.385263 -1.05824 0.0132777 0.0; -1.89572 5.93124 -0.000474582 -0.718598; -6.93677 -0.000366314 -0.00679796 -1.23076]
[-0.0669536 0.21449 -4.63567e-5 -0.0257028; -3.47358 -0.0375131 -0.00339891 -0.752836; 19.9893 2.06523 -0.00701735 3.42185; 1.1688 3.25118 -0.0402998 0.0949883]
[1.1818 0.120343 -0.000389637 0.20317; 0.401327 2.2118 -0.020275 -0.0227716; 4.7063 -8.87201 0.11789 1.45645; 13.8437 -0.328589 0.0133574 3.02077]
[0.399127 -0.784062 0.0103213 0.125084; 10.0848 0.158001 0.00563369 2.05407; -38.7078 -4.30855 0.0113649 -8.15823; -4.82052 -12.1505 0.11012 -0.33318]
[-4.45527 -0.480777 0.00117322 -0.93789; -1.36407 -8.12505 0.0820559 0.160988; -7.83515 32.2765 -0.309769 -3.33768; -72.7968 0.373419 -0.060236 -14.9187]
[-1.03592 4.53738 -0.0436269 -0.455